In [1]:
import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error
import calendar
import os

In [2]:
# MySQL database connection function
def connect_to_database():
    try:
        # Establishing connection to the database
        connection = mysql.connector.connect(
            host='crypto-matter.c5eq66ogk1mf.eu-central-1.rds.amazonaws.com',
            database='Crypto',
            user='Jing',  # Replace with your actual first name
            password='Crypto12!'
        )

        if connection.is_connected():
            db_info = connection.get_server_info()
            print("Connected to MySQL database, MySQL Server version: ", db_info)
            return connection

    except Error as e:
        print("Error while connecting to MySQL", e)
        return None

connection = connect_to_database()

Connected to MySQL database, MySQL Server version:  8.0.39


In [3]:
df_rates = pd.read_csv('../data/rates_data_v4.csv')
print(len(df_rates))
df_rates.head(3)

2141202


,Unnamed: 0,Symbol,Timestamp,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg
0,0,DAI,2020-12-01 16:00:00,0.000899,0.003996,0.083333,0.042083
1,1,DAI,2020-12-01 17:00:00,0.000000,0.000000,0.250000,0.046250
2,2,DAI,2020-12-01 18:00:00,0.000000,0.000000,0.250000,0.046250


In [4]:
# Function to get all table names in the database
def get_table_names(connection):
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    return [table[0] for table in tables]

get_table_names(connection)

[bytearray(b'US_Bond_Yield'),
 bytearray(b'crypto_lending_borrowing'),
 bytearray(b'crypto_price'),
 bytearray(b'crypto_reference')]

In [5]:
symbols = pd.DataFrame(columns = ['matching','rates','prices'])
rates_symbols = df_rates['Symbol'].unique()
symbols['rates'] = rates_symbols
symbols['matching'] = [x.lower() for x in rates_symbols]
symbols.head(3)

,matching,rates,prices
0,dai,DAI,NaN
1,tusd,TUSD,NaN
2,usdc,USDC,NaN


In [9]:
import pandas as pd
import glob
import os 

path = 'prices/*.csv'
csv_files = glob.glob(path)
df_price = pd.concat([
    pd.read_csv(file).assign(symbol=os.path.splitext(os.path.basename(file))[0])
    for file in csv_files
], ignore_index=True)
print(df_price.columns)
print(len(df_price))


Index(['Unnamed: 0', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'close_time', 'quote_asset_volume', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore',
       'symbol'],
      dtype='object')
1187631


In [10]:
prices_symbols = df_price['symbol'].unique()
len(prices_symbols)

38

In [11]:
for price_symbol in prices_symbols:
     x = price_symbol.replace('USDT','')
     x = x.lower()
     if x in symbols['matching'].values:
          index = symbols[symbols['matching'] == x].index
          symbols.loc[index, 'prices'] = price_symbol
symbols.head(3)

,matching,rates,prices
0,dai,DAI,DAIUSDT
1,tusd,TUSD,TUSDUSDT
2,usdc,USDC,USDCUSDT


In [12]:
accpetable_tokens = ['bat', 'link', 'knc', 'mkr', 'mana', 'zrx', 'snx', 'wbtc', 'enj', 'ren', 'yfi', 'uni', 'crv', 'bal', 'ens', '1inch']
len(accpetable_tokens)

16

In [13]:
symbols = symbols[symbols['matching'].isin(accpetable_tokens)]
print(len(symbols))
symbols.head(3)

16


,matching,rates,prices
5,bat,BAT,BATUSDT
6,link,LINK,LINKUSDT
7,knc,KNC,KNCUSDT


In [14]:
df_rates_filtered = df_rates[df_rates['Symbol'].isin(symbols['rates'].to_list())]
df_rates_filtered.reset_index(inplace=True, drop=True)
print(len(df_rates_filtered))

515214


In [16]:
def replace_symbol(symbol):
     return symbols[symbols['rates'] == symbol]['prices'].values[0]
print(replace_symbol('BAT'))

BATUSDT


In [17]:
df_rates_filtered['Symbol'] = df_rates_filtered['Symbol'].apply(replace_symbol)
df_rates_filtered['Symbol'].value_counts(dropna=False)

C:\Users\hgnik\AppData\Local\Temp\ipykernel_43048\129440409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rates_filtered['Symbol'] = df_rates_filtered['Symbol'].apply(replace_symbol)


Symbol
MKRUSDT      34297
WBTCUSDT     34296
UNIUSDT      34293
ENJUSDT      34264
LINKUSDT     34222
SNXUSDT      34203
MANAUSDT     34146
ZRXUSDT      33796
CRVUSDT      33688
YFIUSDT      33631
RENUSDT      33624
BATUSDT      33158
KNCUSDT      32964
BALUSDT      32791
ENSUSDT      23224
1INCHUSDT    18617
Name: count, dtype: int64

In [20]:
df_rates_filtered.replace(0, -50, inplace=True)
count_minus_50 = (df_rates_filtered == -50).sum().sum()
print(count_minus_50)
print(df_rates_filtered.isna().sum())

215011
Unnamed: 0                0
Symbol                    0
Timestamp                 0
liquidityRate_avg         0
variableBorrowRate_avg    0
utilizationRate_avg       0
stableBorrowRate_avg      0
dtype: int64


C:\Users\hgnik\AppData\Local\Temp\ipykernel_43048\1387197028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rates_filtered.replace(0, -50, inplace=True)


In [24]:
df_rates_filtered.reset_index(inplace=True, drop=True)
df_rates_filtered.columns

Index(['Unnamed: 0', 'Symbol', 'Timestamp', 'liquidityRate_avg',
       'variableBorrowRate_avg', 'utilizationRate_avg',
       'stableBorrowRate_avg'],
      dtype='object')

In [29]:
df_rates_filtered.loc[0, 'Timestamp']

'2020-12-02 14:00:00'

In [31]:
# Convert the 'timestamp' column to datetime format, if it's not already
df_rates_filtered['date'] = pd.to_datetime(df_rates_filtered['Timestamp'])

# Format the datetime to MySQL's expected string format
df_rates_filtered['date'] = df_rates_filtered['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

type(df_rates_filtered.loc[0,'date'])

C:\Users\hgnik\AppData\Local\Temp\ipykernel_43048\2526683159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rates_filtered['date'] = pd.to_datetime(df_rates_filtered['Timestamp'])
C:\Users\hgnik\AppData\Local\Temp\ipykernel_43048\2526683159.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rates_filtered['date'] = df_rates_filtered['date'].dt.strftime('%Y-%m-%d %H:%M:%S')


str

In [34]:
cursor = connection.cursor()
unique_symbols = df_rates_filtered['Symbol'].unique()


In [37]:
# Create placeholders for each symbol in unique_symbols
placeholders = ", ".join(["%s"] * len(unique_symbols))
existing_symbols_query = f"SELECT crypto_symbol FROM crypto_reference WHERE crypto_symbol IN ({placeholders})"

# Execute the query with unique_symbols as individual parameters
cursor.execute(existing_symbols_query, unique_symbols)
existing_symbols = {row[0] for row in cursor.fetchall()}


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)' at line 1

In [38]:
insert_statement = """
    INSERT IGNORE INTO crypto_lending_borrowing (crypto_symbol, date, lending_rate, borrowing_rate, utilization_rate, stable_borrow_rate)
    VALUES (%s, %s, %s, %s, %s, %s)
"""

data = df_rates_filtered[['Symbol', 'date', 'liquidityRate_avg', 'variableBorrowRate_avg', 'utilizationRate_avg', 'stableBorrowRate_avg']].values.tolist()
cursor.executemany(insert_statement, data)

# Commit and close connection
connection.commit()
cursor.close()
connection.close()

In [40]:
connection = connect_to_database()
# Function to view data from a specific table
def view_table_data(connection, table_name, limit=300):
    cursor = connection.cursor()
    query = f"SELECT * FROM {table_name} LIMIT %s;"
    cursor.execute(query, (limit,))
    data = cursor.fetchall()
    return data

view_table_data(connection,'crypto_lending_borrowing', limit=30)

Connected to MySQL database, MySQL Server version:  8.0.39


[(1,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 14, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.030000')),
 (2,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 15, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.030000')),
 (3,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 16, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.013598'),
  Decimal('0.030000')),
 (4,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 17, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.014834'),
  Decimal('0.030000')),
 (5,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 18, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.014834'),
  Decimal('0.030000')),
 (6,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 19, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.014834'),
  Decimal('0.030000')),
 (7,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 20, 0),
 